In [ ]:
# default_exp core

# Implementation

This is the implementation of `ndpretty`. 

In [ ]:
#export
import numpy as np
from sys import modules

import IPython
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Some static definitions of the colour scale and the number format.

In [ ]:
#export
lowest_color = (110, 110, 255)
highest_color = (220, 55, 55)

number_format = '%.5g'

# if an array dimension is bigger than this, automatically reduce by slicing to improve performance
default_max_dim_size = 200

And the core implementation:

In [ ]:
#export
def ndarray_html(a):
    assert type(np.array([])) == np.ndarray, 'Only numpy ndarrays are supported'

    if len(a.shape) == 1:
        if a.shape[0] == 0:
            print('[]')
            return
        a = a[:,np.newaxis]

    slice_str = _create_default_slice(a)
    slice_widget = widgets.Text(
        value=slice_str,
        placeholder="e.g. " + slice_str,
        description='Slice:',
        disabled=False
    )
    interact(_html_slice_array, a=fixed(a), slice_str=slice_widget)

def _create_default_slice(a):
    d = len(a.shape)
    assert d >= 2
    sl = []
    for i in [0, 1]:
        sl.append(":" if min(a.shape[i], default_max_dim_size) < default_max_dim_size else ":" + str(default_max_dim_size))
    sl.extend(["0"] * (d - 2))
    return "[" + ", ".join(sl) + "]"


def _to_HTML(a, alphas, is_numeric, lowest_color, highest_color):
    html = '<div style="overflow: auto">'
    html += '<table style="border-spacing: 0px;">'
    html += '<tr>'
    html += '<th></th>'
    for j in range(a.shape[1]):
        html += f'<th>{j}</th>'
    html += '</tr>'

    for i in range(a.shape[0]):
        html += "<tr>"
        html += f'<td><b>{i}</b></td>'
        for j in range(a.shape[1]):
            alpha = alphas[i][j]
            rgb_color = tuple((alpha * np.array(highest_color) + (1 - alpha) * np.array(lowest_color)).astype(np.uint8))
            hex_color = '#%02x%02x%02x' % rgb_color
            if is_numeric:
                value = number_format % a[i][j]
            else:
                value = str(a[i][j])
            html += '<td style="background-color: %s; padding:5px !important">%s</td>' % (hex_color, value)
        html += "</tr>"
    html += '</table>'
    html += '</div>'
    return html

def _html_slice_array(a, slice_str):
    try: 
        sliced_a = eval("a" + slice_str)
        assert len(sliced_a.shape) == 2, "didn't slice down to 2D"
        _html_array(sliced_a)
    except Exception as e:
        print("Invalid slice: " + str(e))

def _html_array(a):   
    is_numeric = np.issubdtype(a.dtype, np.number)

    if is_numeric:
        a_range = a.max() - a.min()
        if a_range == 0:
            alphas = np.zeros_like(a, dtype=np.float) + 0.5
        else:
            alphas = (a - a.min()) / a_range
    elif np.issubdtype(a.dtype, np.bool_):
        alphas = a.astype(np.float)
    else:
        alphas = np.zeros_like(a, dtype=np.float) + 0.5

    html = _to_HTML(a, alphas, is_numeric, lowest_color, highest_color)
    IPython.core.display.display(IPython.display.HTML(html))

## Example usages

A numeric 2D array:

In [ ]:
ndarray_html(np.diag(np.ones(10)))

interactive(children=(Text(value='[:, :]', description='Slice:', placeholder='e.g. [:, :]'), Output()), _dom_c…

A numeric 1D array:

In [ ]:
ndarray_html(np.ones((10,)))

interactive(children=(Text(value='[:, :]', description='Slice:', placeholder='e.g. [:, :]'), Output()), _dom_c…

A numeric 4D array:

In [ ]:
ndarray_html(np.random.rand(2, 3, 4, 5))

interactive(children=(Text(value='[:, :, 0, 0]', description='Slice:', placeholder='e.g. [:, :, 0, 0]'), Outpu…

When an ndarray has more than two dimensions, a text field (an [ipywidget](https://ipywidgets.readthedocs.io)) is depicted on top of the table, where the user can slice down the array to have two dimension, so that it can be displayed in a table.

_Note: The `ipywidget` that is used for multi-dimensional arrays is not correctly rendered in the documentation. Therefore see this screenshot:_ 

<img src="img/4D.png" height="80px" />

A string array:

In [ ]:
ndarray_html(np.diag(['nd', 'pretty', 'ndpretty']))

interactive(children=(Text(value='[:, :]', description='Slice:', placeholder='e.g. [:, :]'), Output()), _dom_c…

A bool array:

In [ ]:
ndarray_html(np.array([True, False, True]))

interactive(children=(Text(value='[:, :]', description='Slice:', placeholder='e.g. [:, :]'), Output()), _dom_c…

### Automatic default slicing

When an array would generally be too huge to display, it is automatically downsliced to the value of `default_max_dim_size`.

In [ ]:
print(f"default_max_dim_size is {default_max_dim_size}")
ndarray_html(np.random.rand(10000, 1000))

default_max_dim_size is 200


interactive(children=(Text(value='[:200, :200]', description='Slice:', placeholder='e.g. [:200, :200]'), Outpu…

## Registering formatters for IPython

We don't always want to call `ndarray_html` to show our nice table. In order to make it the default formatter for cell return values, here are some helper functions to automatically register the formatters.

This makes use of [IPyhton third-party formatters](https://ipython.readthedocs.io/en/stable/config/integrating.html?highlight=third%20party#formatters-for-third-party-types).

In [ ]:
#export
def register_formatter(dtype, html_formatter, print_formatter=None):
    formatters = get_ipython().display_formatter.formatters
    
    formatters['text/html'].for_type(dtype, html_formatter)
    if print_formatter is not None:
        formatters['text/plain'].for_type(dtype, print_formatter)

### for `ndarray`

In [ ]:
#export
def ndarray_stats_print_formatter(x, _, __):
    print('×'.join(map(str, x.shape)) + " " + str(x.dtype) + ' ndarray')

def no_print_formatter(x, _, __):
    return

def register_ndarray_formatter(print_formatter=ndarray_stats_print_formatter):
    register_formatter(np.ndarray, ndarray_html, print_formatter)

A formatter can be registered like this:

In [ ]:
register_ndarray_formatter()

From then onwards, all returned ndarrays are formatted using ndpretty.

In [ ]:
np.random.rand(2, 8)

2×8 float64 ndarray


interactive(children=(Text(value='[:, :]', description='Slice:', placeholder='e.g. [:, :]'), Output()), _dom_c…

### for `torch.Tensor`

We also define a default formatter for PyTorch `Tensor`s:

In [ ]:
#export
def torch_tensor_html(t):
    ndarray_html(t.numpy())

def tensor_stats_print_formatter(x, _, __):
    print('×'.join(map(str, x.shape)) + " " + str(x.dtype) + ' tensor')

def register_torch_tensor_formatter(print_formatter=tensor_stats_print_formatter):
    register_formatter(torch.Tensor, torch_tensor_html, print_formatter)

In [ ]:
if 'torch' in modules:
    register_torch_tensor_formatter()
    t = torch.Tensor(np.random.rand(10, 4))
else:
    t = 'torch not imported'
t

'torch not imported'

### Default configuration for convenience

Finally we define a convenience function to quickly initialise the default configuration where both ndarrays and PyTorch tensors are formatted.

In [ ]:
#export
def default():
    register_ndarray_formatter()
    if 'torch' in modules:
        register_torch_tensor_formatter()

In [ ]:
default()

---

### TODOs and potential new features
- [x] zero-dim arrays
- [x] strings
- [x] ints
- [x] bools
- [x] size limit: if array is larger than certain size, do auto-slicing
- [ ] slider for decimal places or text field for format
- [ ] fallback in case of exception
- [ ] unregister hook
- [ ] check if in IPython and don't crash otherwise